In [1]:
import random
import numpy as np
import random
import pickle
import letor_metrics
import pyximport
import sys
from tqdm import tqdm
pyximport.install()
import matplotlib


raw_data_train = np.loadtxt('/data/recnet_draft/ml100k/train_all_raw.csv', skiprows = 1, delimiter=',')
raw_data_test = np.loadtxt('/data/recnet_draft/ml100k/test_all_raw.csv', skiprows = 1, delimiter=',')
raw_data = np.concatenate((raw_data_train, raw_data_test))
from dataset_tt_static import TripletsDataset

ds = TripletsDataset(raw_data_train, raw_data_test, threshold_user=60, rnd_seed=42)
ds.train_test_split()

ds.init_cached_random()
import tensorflow as tf
import bprnn_ml100k_param_tune_all
import imp

N_USERS = int(max(raw_data[:, 0])) + 1
N_ITEMS = int(max(raw_data[:, 1])) + 1
N_EMBEDDINGS = 17

import tensorflow.contrib.slim as slim
imp.reload(bprnn_ml100k_param_tune_all)

#%%
def inner_network(user_emb, item_emb):
    joined_input = tf.concat([user_emb, item_emb], 1)
    net = slim.fully_connected(inputs=joined_input, num_outputs=32, activation_fn=tf.nn.relu)
#     net = slim.fully_connected(inputs=joined_input, num_outputs=64, activation_fn=tf.nn.relu)
#     net = slim.dro
    net = slim.fully_connected(inputs=net, num_outputs=1, activation_fn=None)
    return net

model = bprnn_ml100k_param_tune_all.BPR_NN(N_USERS, N_ITEMS, N_EMBEDDINGS, alpha=1, beta=0, alpha_reg=0.0001, inner_net=inner_network)
model.build_graph()
model.initialize_session()

losses = []
batch_size = 512
for n_batches, cur_optim in [(10000, model.trainer_3)]:
    for i in tqdm(range(n_batches)):
        if i%1000==0 and i/1000 >= 1:
            model.alpha = model.alpha - 0.1
            model.beta = model.beta + 0.1
            print('batch number:'+str(i))
            print('alpha:'+ str(model.alpha))
            print('beta:'+ str(model.beta))
        batch = ds.sample_train_batch(n_samples=batch_size)
        fd = {
            model.user_ids:  batch['users'], 
            model.left_ids:  batch['left_items'],
            model.right_ids: batch['right_items'],
            model.target_y:  batch['y'],
        }
        el, nl, reg, t, _ = model.session.run(
            [model.embedding_loss, model.net_loss, model.regularization, model.target, cur_optim], 
            feed_dict=fd
        )
        losses.append((el, nl, reg, t))
        if i%1000==0:
            user_norm = np.linalg.norm(model.weights_u)
            item_norm = np.linalg.norm(model.weights_i)
            print('[it {}] weight norms, users: {}, items: {}'.format(i, user_norm, item_norm))
            print('[it {}] metrics (emb_loss, net_loss, reg, target): {}'.format(i, losses[-1]))

Split users:   5%|▌         | 21/408 [00:00<00:01, 208.49it/s]

  0%|          | 13/10000 [00:00<29:51,  5.57it/s]

[it 0] weight norms, users: 72.81585693359375, items: 96.86475372314453
[it 0] metrics (emb_loss, net_loss, reg, target): (0.76785225, 0.69265568, 16.98176, 0.69435388)


 10%|█         | 1027/10000 [00:07<00:59, 149.94it/s]

batch number:1000
alpha:0.9
beta:0.1
[it 1000] weight norms, users: 73.4916763305664, items: 97.24102020263672
[it 1000] metrics (emb_loss, net_loss, reg, target): (0.7375989, 0.46065721, 17.529514, 0.46241015)


...

 80%|████████  | 8024/10000 [00:59<00:13, 148.54it/s]

batch number:8000
alpha:0.20000000000000015
beta:0.7999999999999999
[it 8000] weight norms, users: 76.76631927490234, items: 102.27267456054688
[it 8000] metrics (emb_loss, net_loss, reg, target): (0.79241133, 0.2155387, 19.784639, 0.21751715)


 90%|█████████ | 9032/10000 [01:06<00:06, 158.11it/s]

batch number:9000
alpha:0.10000000000000014
beta:0.8999999999999999
[it 9000] weight norms, users: 77.02434539794922, items: 102.82274627685547
[it 9000] metrics (emb_loss, net_loss, reg, target): (0.83365059, 0.20401697, 20.114483, 0.20602842)


100%|██████████| 10000/10000 [01:12<00:00, 137.39it/s]


In [3]:
export_basename = '/data/recnet_draft/ml100k/dec_alpha/vectors/'
export_pred = open(export_basename + 'pr_ml100k_10', 'w')
export_true = open(export_basename + 'gt_ml100k_10', 'w')

ndcg_vals = []
for u in tqdm(ds.data_keys, desc='Prediction', leave=True):
    # if not u in ds.test:
    if not u in ds.test or not ds.test[u]:
        continue
    response = np.zeros(len(ds.test[u]))
    fd = {
            model.user_ids:  (np.ones(len(ds.test[u]))*u).astype(np.int32), 
            model.left_ids:  np.array([i for (i, r) in ds.test[u]]).astype(np.int32),
        }
    response += model.session.run(model.embedding_left, feed_dict=fd)[:, 0]
    response += model.session.run(model.left_output, feed_dict=fd)[:, 0]

    # make relevances
    relevances = np.array([r for (i, r) in ds.test[u]])
    items = np.array([i for (i, r) in ds.test[u]])  # it's already sorted by true relevance
    itemsGroundTruth = np.array([i for (i,r) in ds.test[u] if r == 1])
    predicted_ranking = np.argsort(-response)

    # write down predictions
    export_pred.write(' '.join(map(str, [u] + list(items[predicted_ranking]))) + '\n')
    export_true.write(' '.join(map(str, [u] + list(itemsGroundTruth))) + '\n')

    # calc score
    gain = letor_metrics.ndcg_from_ranking(relevances, predicted_ranking, 10)
    ndcg_vals.append(gain)


# In[32]:

print(ndcg_vals)

Prediction: 100%|██████████| 408/408 [00:00<00:00, 3042.85it/s]

[0.8396538878217249, 0.42264158484677827, 0.70150996472324456, nan, 1.0, 0.94039623903753067, 0.61776399916124758, 1.0, 0.5, 0.85034490553475461, 1.0, 0.93637921180104833, 0.45560514958746035, 0.38685280723454163, 0.87060868542699299, 0.93056877806322291, 0.63092975357145753, 0.91972078914818756, 0.77209115442140641, 0.66650486981001134, 0.94690242952597448, 0.72641273023066366, 0.73757284928660216, 0.85999191954790177, 0.48247555939075504, 0.96746798348916929, 0.50148397124678246, nan, 1.0, 0.92020546143755177, 0.36334128725957199, 0.70263051638546459, 1.0, 0.84860265890399011, 1.0, 0.87381008988013231, 1.0, 0.9216017310213247, 0.69759616950614933, 0.58679458467691459, 1.0, 0.99332229988146181, 1.0, 0.7702043899253147, nan, 0.97992148014470837, nan, nan, 0.63092975357145753, 0.56191072411498288, 0.77990823370191997, 0.55226547738705301, 0.88995411685096004, 1.0, 0.43067655807339306, 0.62405052000383787, 1.0, 0.87721531533804931, 0.63092975357145753, 1.0, 0.88397641507283109, nan, nan,

In [ ]:
from matplotlib.pyplot import *

In [ ]:
import matplotlib.pyplot as plt
ints = [100, 500, 1000, 2000, 5000, 10000]
pos = 1
fig, ax = plt.subplots(nrows = 3, ncols = 2)
#ax[-1, -1].axis('off')

for int in ints:
    axis = plt.subplot(3, 2, pos)
    axis.plot([x[0] for x in losses[:int]], c='b', label='embedding_loss')
    grid()
    legend()
    xlabel('n_batches')
    ylabel('embedding loss')
    yscale('log')
    draw()
    show(block=False)
    pos = pos + 1
    fig.savefig('/home/sumit/recnet_draft/plots/ml100k/embedding_loss_dec_alpha.pdf')
plt.close()

In [ ]:
import matplotlib.pyplot as plt
ints = [100, 500, 1000, 2000, 5000, 10000]
pos = 1
fig, ax = plt.subplots(nrows=3, ncols=2)

for int in ints:
    axis = plt.subplot(3, 2, pos)
    axis.plot([x[1] for x in losses[:int]], c='m', label='ranking_loss')
    grid()
    legend()
    xlabel('n_batches')
    ylabel('logloss')
    yscale('log')
    draw()
    show(block=False)
    pos = pos + 1
    fig.savefig('/home/sumit/recnet_draft/plots/ml100k/ranking_loss_dec_alpha.pdf')
plt.close()

In [ ]:
import matplotlib.pyplot as plt
ints = [100, 500, 1000, 2000, 5000, 10000]
pos = 1
fig, ax = plt.subplots(nrows=3, ncols=2)
for int in ints:
    axis = plt.subplot(3, 2, pos)
    axis.plot([x[3] for x in losses[: int]], c='g', label='target')
    grid()
    legend()
    xlabel('n_batches')
    ylabel('recnet loss')
    #yscale('log')
    draw()
    show(block=False)
    pos = pos + 1
    fig.savefig('/home/sumit/recnet_draft/plots/ml100k/target_loss_dec_alpha')
plt.close()
    

In [ ]:
plt.close("all")